# Importing dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
#cleaning text data
import nltk
import numpy as np
import re #regular expressions(rmeoving fullstops etc)
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91738\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset= pd.read_csv('train.csv',encoding = "ISO-8859-1",nrows=7000)
#dataset.drop(["date","day","topic","name"], axis = 1,inplace=True)

In [3]:
#dataset.drop(dataset[dataset['sentiment'] ==2].index, inplace = True)

In [4]:
#dataset.sentiment =dataset.sentiment.replace(4,1)

# removing punctuation and number

In [5]:
def preprocess_tweet(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet.lower()
    #convert all urls to sting "URL"
    tweet= re.sub('[^a-zA-Z]',' ',tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet
dataset['text'] = dataset['text'].apply(preprocess_tweet)

In [6]:
c=[]
for i in range(0,7000):
    review1= re.sub('[^a-zA-Z]',' ',dataset['text'].iloc[i])
    #cover it as lower case
    review1=review1.split()#splits words and forms as list
    ps= PorterStemmer()
    review1=[ps.stem(word) for word in review1 if not word in set(stopwords.words('english'))]
    review1=' '.join(review1)
    c.append(review1)
#print(c)

In [7]:
#creating a bag of words model

from  sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=3000)
x= cv.fit_transform(c).toarray()
y= dataset.iloc[:,0:1].values

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
pickle.dump(cv.vocabulary_,open("cf.pkl","wb"))

# importing libraries for the model

In [9]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()

Using TensorFlow backend.
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

# Adding Input layer

In [10]:
model.add(Dense(input_dim=x.shape[1],init="random_uniform",activation="sigmoid",output_dim=100))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=3000, activation="sigmoid", units=100, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# Adding hidden layer

In [11]:
model.add(Dense(init="random_uniform",activation="sigmoid",output_dim=20))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=20, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# Adding output layer

In [12]:
model.add(Dense(output_dim=1,init='random_uniform',activation='sigmoid'))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# configure the learning process

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#optimizing model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
model.fit(x_train,y_train,epochs=50,batch_size=130)
y_pred=model.predict(x_test)
#print(x_test)
y_pred=(y_pred>0.5)
model.save('final1.h5')


Epoch 1/50
4900/4900 [==============================] - 0s 90us/step - loss: 0.6680 - accuracy: 0.6324
Epoch 2/50
4900/4900 [==============================] - 0s 53us/step - loss: 0.6572 - accuracy: 0.6324
Epoch 3/50
4900/4900 [==============================] - 0s 54us/step - loss: 0.6556 - accuracy: 0.6324
Epoch 4/50
4900/4900 [==============================] - 0s 53us/step - loss: 0.6536 - accuracy: 0.6324
Epoch 5/50
4900/4900 [==============================] - 0s 53us/step - loss: 0.6514 - accuracy: 0.6324
Epoch 6/50
4900/4900 [==============================] - 0s 53us/step - loss: 0.6480 - accuracy: 0.6324
Epoch 7/50
4900/4900 [==============================] - 0s 53us/step - loss: 0.6428 - accuracy: 0.6324
Epoch 8/50
4900/4900 [==============================] - 0s 52us/step - loss: 0.6338 - accuracy: 0.6324
Epoch 9/50
4900/4900 [==============================] - 0s 52us/step - loss: 0.6174 - accuracy: 0.6343
Epoch 10/50
4900/4900 [==============================] - 0s 52us/step - 

# confusion matrix for accuracy

In [17]:
#confusion matrix
#accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
loaded_vec =CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("cf.pkl", "rb")))
da=""
da=da.split("delimiter")
result= model.predict(loaded_vec.transform(da))


# own prediction

In [16]:
r=model.predict(cv.transform(["hate"]))
print("first predicton",r)

first predicton [[0.00763537]]
